W poprzednich dwóch zadaniach ekstrahowaliśmy cechy z sygnału podzielonego na ramki, uzyskując obraz (macierz 2D) opisujący sygnał. Jest to bardzo często stosowane w analizie sygnałów podejście, ale nie jedyne.

Z sygnałów można też ekstrahować inne parametry, np. parametry czasowe sygnału (częstotliwość przejść przez zero, środek ciężkości sygnału), parametry widmowe (momenty widmowe, kurtoza i skośność widma), parametry formantowe (częstotliwości formantów, stosunek ich amplitud) i wiele innych. Zbiór różnych parametrów sygnałów, które dają bardzo dobre rezultaty w klasyfikacji sygnałów akustycznych jest zawarty w bibliotece OpenSMILE.

Jeżeli użyjemy najnowszej wersji tej bilbioteki do ekstrakcji cech, uzyskamy  6373 parametry opisujące pojedynczy sygnał (osoby zainteresowane mogę doczytać, jakie dokładnie są to cechy https://audeering.github.io/opensmile/about.html)

In [ ]:
import opensmile
import numpy as np
import pandas as pd
import time
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

feats = smile.process_file('dane_testowe/1-phrase.wav') #feats od ang. features - cechy
feats1 = smile.process_file('dane_testowe/10-phrase.wav')
feats = pd.concat([feats, feats1])
feats

Ponownie wykorzystując napisany wcześniej kod, wylicz parametry wszystkich sygnałów.

In [ ]:
import librosa 

from os import listdir
pliki = listdir('dane_testowe/')
pliki = ['dane_testowe/'+x for x in pliki]
Phrases = []
pliki.sort()
#for i in pliki:
#    x, fs = librosa.load(i)
#   data = x
#    a = data.flatten()
#    a = a.transpose()
#    Phrases.append(a)
#min_length = min(map(lambda x: x.shape[-1], Phrases))
#Phrases = np.stack([x[..., :min_length] for x in Phrases])
feats = []
for i in pliki:
    feats.append(smile.process_file(i))
feats = pd.concat(feats)
feats.shape

Przed przystąpieniem do selekcji cech i uczenia klasyfikatora, dane należy najpierw ustandaryzować. Można to zrobić np. używając funkcji StandardScaler z bilbioteki sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

Zachowujemy kolejność:
1. wyznaczenie średniej i wariancji/odchylenia standardowego zbioru uczącego, które będą wykorzystane do standaryzacji
2. standaryzacja zbioru uczącego: 
3. standaryzacja zbioru testowego w oparciu o parametry ze zbioru testowego (czyli średnia i wariancja/odchylenie standardowe używane do standaryzacji zbioru testowego są wyliczane na zbiorze uczącym - w ten sposób wszystkie dane są przekształcane jednakowo): 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
labels = []
for i in pliki:
    if i.__contains__('phrase'):
        labels.append(1)
        continue
    labels.append(0)
X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#print(labels)
#print(pliki)

Na ustandaryzowanych danych wytrenuj klasyfikator i wylicz metryki.

In [ ]:
from sklearn import tree

scaler = StandardScaler()
def test():
    X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
    #scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train, Y_train)
    return clf.predict(X_test), Y_test

good = 0
bad = 0
test_size = 100
tic = time.perf_counter()
for i in range(0, test_size):
    p, t = test()
    for j in range(0, len(p)):
        if p[j] == t[j]:
            good=good+1
            continue
        bad = bad+1
toc = time.perf_counter()
print('sukces: '+ str(good/(good+bad)))
print(' time: '+ str(toc-tic))

6373 cechy opisujące jeden sygnał to dość dużo. Żeby zmniejszyć liczbę tych cech należy zastosować jedną z metod redukcji wymiarowości. Wpłynie to na zmniejszenie skomplikowania niektórych rodzajów klasyfikatorów, przyspieszy proces uczenia i pozwoli zmniejszyć przeuczenie modelu.

Zaczniemy od metody bazującej na wartości F z ANOVY. W bibliotece sklearn jest to funkcja SelectKBest(): https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest

Wybierz liczbę cech, które chcesz zachować. Funkcja wybierze ze wszystkich 6373 cech tylko tyle, ile sprecyzujesz w taki sposób, by jak najlepiej opisywać zmienność i różnicować obiekty.

Liczbę cech można optymalizować, ale tym zajmiemy się kiedy indziej. Dzisiaj spróbuj ręcznie dobrać ich liczbę tak, by wyniki klasyfikacji były zadowalające.

In [ ]:
from sklearn.feature_selection import SelectKBest
X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)

liczba_cech = 50
selector = SelectKBest(k=liczba_cech)
#X_train
X_train_Kbest = selector.fit_transform(X_train, Y_train)
X_test_Kbest = selector.transform(X_test)

Wytrenuj klasyfikator na zredukowanej liczbie cech. Jakie wartości metryk wyszły teraz? Czy są gorsze niż wcześniej?

In [ ]:

def test_k():
    X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
    #selector = SelectKBest(k=10)
    X_train_Kbest = selector.fit_transform(X_train, Y_train)
    X_test_Kbest = selector.transform(X_test)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train_Kbest, Y_train)
    return clf.predict(X_test_Kbest), Y_test

good = 0
bad = 0
test_size = 100
tic = time.perf_counter()
for i in range(0, test_size):
    p, t = test_k()
    for j in range(0, len(p)):
        if p[j] == t[j]:
            good=good+1
            continue
        bad = bad+1
toc = time.perf_counter()
print('sukces: '+ str(good/(good+bad)))
print(' time: '+ str(toc-tic))

Kolejną funkcją wartą poznania jest metoda bardzo podobna do SelectKBest, czyli SelectPercentile. Działa na tej samej zasadzie, ale zamiast określać konkretną liczbę cech, które chcemy zachować, określamy jaka część cech ma być zachowana, np. 0.4 (40%).

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile

In [ ]:
from sklearn.feature_selection import SelectPercentile
X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)

percentile = 0.2
selector = SelectPercentile(percentile=percentile)
#X_train
X_train_perc = selector.fit_transform(X_train, Y_train)
X_test_perc = selector.transform(X_test)

def test_p():
    X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
    #selector = SelectPercentile(percentile=percentile)
    X_train_perc = selector.fit_transform(X_train, Y_train)
    X_test_perc = selector.transform(X_test)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train_perc, Y_train)
    return clf.predict(X_test_perc), Y_test

good = 0
bad = 0
test_size = 100
tic = time.perf_counter()
for i in range(0, test_size):
    p, t = test_p()
    for j in range(0, len(p)):
        if p[j] == t[j]:
            good=good+1
            continue
        bad = bad+1
toc = time.perf_counter()
print('sukces: '+ str(good/(good+bad)))
print(' time: '+ str(toc-tic))


Ostatnią metodą, którą dzisiaj poznamy jest rekursywna eliminacja cech (RFE, ang. recursive feature elimination). Polega na tym, że estymator początkowo uczony jest na całym zbiorze danych i tworzony jest ranking cech w oparciu o to, jak bardzo są ważne podczas estymacji. Cechy najmniej istotne są usuwane i cały proces jest powtarzany. Usuwanie cech trwa tak długo, aż uzyskamy taką liczbę, jak sprecyzowana.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE

In [173]:

X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
liczba_cech = 5
step = 100
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=liczba_cech, step=step) #step - jeżeli >=1, to jest to liczba cech do 
                                                                    #usunięcia w danej iteracji, jeżeli <0 - 
                                                                    #część cech do usunięcia
X_train_RFE = selector.fit_transform(X_train, Y_train)
X_test_RFE = selector.transform(X_test)

def test_RFE():
    X_train, X_test, Y_train, Y_test = train_test_split(feats, labels, test_size=0.2)
    #estimator = LinearRegression()
    #selector = RFE(estimator, n_features_to_select=5, step=10)
    X_train_RFE = selector.fit_transform(X_train, Y_train)
    X_test_RFE = selector.transform(X_test)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X_train_RFE, Y_train)
    return clf.predict(X_test_RFE), Y_test

good = 0
bad = 0
test_size = 50
tic = time.perf_counter()
for i in range(0, test_size):
    p, t = test_RFE()
    for j in range(0, len(p)):
        if p[j] == t[j]:
            good=good+1
            continue
        bad = bad+1
toc = time.perf_counter()
print('sukces: '+ str(good/(good+bad)))
print(' time: '+ str(toc-tic))

sukces: 0.95
 time: 38.64269279999917


Powtórz proces uczenia i predykcji na cechach uzyskanych metodą RFE. Jakie wartości metryk wyszły tym razem? 

Za tydzień omówimy jeszcze PCA i ICA, które też mogą być wykorzystywane do redukcji wymiarowości. Metod jest oczywiście więcej, ale nie zdążymy przerobić wszystkich. Dla zainteresowanych: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection